#     **SPOTIFY MOOD PREDICTION AND MUSIC RECOMMENDATION SYSYEM** 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import numpy as np
import os
import pandas as pd
# To make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import seaborn as sns
#set style of plots
sns.set_style('white')

#define a custom palette
customPalette = ['#630C3A', '#39C8C6', '#D3500C', '#FFB139']
sns.set_palette(customPalette)
sns.palplot(customPalette)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
path='/content/drive/MyDrive/spotify/data.csv'

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/spotify/data.csv')

In [ ]:
data

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
s=data.groupby("artists")["popularity"].sum().sort_values(ascending=False).head(170652)


In [ ]:
"s.head(10)

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(data.corr(),annot=True)

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
cols = ["popularity","acousticness","instrumentalness","speechiness","danceability" ]
sns.pairplot(data[cols], height = 2.5 )
plt.show();

In [ ]:
data.info()

# **FEATURE SELECTION**

In [ ]:

features = data[['danceability','liveness','acousticness','loudness','speechiness','valence','energy']]


DATA PREPROCESSING

In [ ]:
from sklearn import preprocessing

loudness = features[['loudness']].values
min_max_scaler = preprocessing.MinMaxScaler()
loudness_scaled = min_max_scaler.fit_transform(loudness)
data['loudness'] = pd.DataFrame(loudness_scaled)




features.hist(bins=50,figsize=(20,15))

**K NEAREST NEIGHBOR ALGORITHM**

In [ ]:
from sklearn.cluster import KMeans

sse = {}
K = range(1,12)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(features)
    sse[k]=km.inertia_

In [ ]:

plt.xlabel('k')
plt.ylabel('sse')
plt.title('Elbow Method To Find k')
sns.pointplot(x=list(sse.keys()),y=list(sse.values()))

OPTIMAL VALUE OF K IS 3

#**CLUSTERING AND MOOD PREDICTION**





In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(features)
labels=kmeans.labels_

In [ ]:
rfm=data.assign(Cluster=labels)
group=rfm.groupby(["Cluster"])
group.agg({
    
    'danceability':'mean',
    'loudness':'mean',
     'valence':'mean',
    'acousticness':['mean','count'],
   

}).round(1)

In [ ]:
rfm_melt=pd.melt(rfm.reset_index(), id_vars =['artists','Cluster'],value_vars=['danceability','loudness','acousticness','valence'],var_name='Metric',value_name='Value')


In [ ]:

plt.xlabel('Metric')
plt.ylabel('Value')
plt.title('snake plot')
sns.pointplot(data=rfm_melt,x='Metric',y='Value',hue='Cluster')

cluster 0--->sad,depreesed


cluster 1--->cheerful,happy


cluster 2--->normal happy mood

In [ ]:
rfm[rfm['Cluster']==1]

In [ ]:
rfm[rfm['Cluster']==2]

In [ ]:
rfm[rfm['Cluster']==0]

# **MUSIC RECOMMENDATION**

In [ ]:
def recommend(n):
  print(rfm[rfm['Cluster']==n])



col=['name','Cluster']
n=rfm.name.to_list()
for i in n:
  i=i.upper()
v=rfm.Cluster.to_list()
while(1):
  print('Enter a song name:\n')
  song=input()
  song.upper()
  if song in n:
    print('Song found, generating recommendations:\n')
    song_index = n.index(song)
    sv=v[song_index]
    recommend(sv)
  else:
    print('Please enter a valid song name:\n')
    continue



  print("Do you want to type another song? (Y/N):\n")
  ans = input()
  ans.upper()
  if(ans=='N'):
    break
  else:
    continue
